In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
os.chdir('E:\НИР\Данные')

Данные брал из следующих источников:
закладки - bookmarks
избранное - favorites
покупки - orders и order_line
посещение - visit
ИКПП - таблицы content_holders и new_stats
bookslog

In [17]:
books_log = pd.read_csv('books_log.csv',sep=';')
bookmarks = pd.read_csv('bookmarks.csv',sep=',')
favorites = pd.read_csv('favorites.csv',sep=';')
orders = pd.read_csv('orders.csv',sep=',')
contents_disciplines = pd.read_csv('contents_disciplines.csv',sep=';')
visit = pd.read_csv('visits.csv',sep=',')
book_shelf = pd.read_csv('book_shelf.csv',sep=',')

In [18]:
bookmarks['ebs_user_id'] = bookmarks['user_id']
book_shelf['ebs_user_id'] = book_shelf['reader_id']

In [19]:
books_log = books_log.astype(str)
bookmarks = bookmarks.astype(str)
favorites = favorites.astype(str)
orders = orders.astype(str)
contents_disciplines = contents_disciplines.astype(str)
visit = visit.astype(str)
book_shelf = book_shelf.astype(str)

Добавляю к данным номер дисциплины по content_id

In [20]:
books_log_1= pd.merge(books_log,contents_disciplines,on='content_id',how='left')
orders = pd.merge(orders,contents_disciplines,on='content_id',how='left')
bookmarks = pd.merge(bookmarks,contents_disciplines,on='content_id',how='left')
favorites= pd.merge(favorites,contents_disciplines,on='content_id',how='left')
visit= pd.merge(visit,contents_disciplines,on='content_id',how='left')
book_shelf = pd.merge(book_shelf,contents_disciplines,on='content_id',how='left')

Считаю количество дисциплин у каждого преподавателя по каждой дисциплине

In [21]:
books_count = books_log_1.groupby(['ebs_user_id','discipline_id'],axis=0).agg(
c_books = pd.NamedAgg(column = 'discipline_id', aggfunc = 'count')
).reset_index()
orders_count = orders.groupby(['ebs_user_id','discipline_id'],axis=0).agg(
c_orders = pd.NamedAgg(column = 'discipline_id', aggfunc = 'count')
).reset_index()
bookmarks_count = bookmarks.groupby(['ebs_user_id','discipline_id'],axis=0).agg(
c_bookmarks = pd.NamedAgg(column = 'discipline_id', aggfunc = 'count')
).reset_index()
favorites_count = favorites.groupby(['ebs_user_id','discipline_id'],axis=0).agg(
c_favourites = pd.NamedAgg(column = 'discipline_id', aggfunc = 'count')
).reset_index()
bookshelf_count = book_shelf.groupby(['ebs_user_id','discipline_id'],axis=0).agg(
c_bookshelf = pd.NamedAgg(column = 'discipline_id', aggfunc = 'count')
).reset_index()
visit_count = visit.groupby(['ebs_user_id','discipline_id'],axis=0).agg(
c_visit = pd.NamedAgg(column = 'discipline_id', aggfunc = 'count')
).reset_index()

Нормирую данные

In [22]:
from sklearn.preprocessing import minmax_scale
books_count['c_books_norm'] = books_count.groupby('discipline_id').c_books.transform(lambda x: minmax_scale(x.astype(float)))
orders_count['c_ordrs_norm'] = orders_count.groupby('discipline_id').c_orders.transform(lambda x: minmax_scale(x.astype(float)))
bookmarks_count['c_bookmarks_norm'] = bookmarks_count.groupby('discipline_id').c_bookmarks.transform(lambda x: minmax_scale(x.astype(float)))
favorites_count['c_favourites_norm'] = favorites_count.groupby('discipline_id').c_favourites.transform(lambda x: minmax_scale(x.astype(float)))
bookshelf_count['c_bookshelf_norm'] = bookshelf_count.groupby('discipline_id').c_bookshelf.transform(lambda x: minmax_scale(x.astype(float)))
visit_count['c_visit_norm'] = visit_count.groupby('discipline_id').c_visit.transform(lambda x: minmax_scale(x.astype(float)))

Объединяю данные в одну таблицу

In [23]:
book_orders = pd.merge(books_count,orders_count,on=['ebs_user_id','discipline_id'],how='outer')
book_orders = pd.merge(book_orders,bookmarks_count,on=['ebs_user_id','discipline_id'],how='outer')
book_orders = pd.merge(book_orders,favorites_count,on=['ebs_user_id','discipline_id'],how='outer')
book_orders = pd.merge(book_orders,bookshelf_count,on=['ebs_user_id','discipline_id'],how='outer')
book_orders = pd.merge(book_orders,visit_count,on=['ebs_user_id','discipline_id'],how='outer')

Заполняю пропущенные значения нулями

In [24]:
book_orders['c_books_norm'] = book_orders['c_books_norm'].fillna(0)
book_orders['c_ordrs_norm'] = book_orders['c_ordrs_norm'].fillna(0)
book_orders['c_bookmarks_norm'] = book_orders['c_bookmarks_norm'].fillna(0)
book_orders['c_favourites_norm'] = book_orders['c_favourites_norm'].fillna(0)
book_orders['c_bookshelf_norm'] = book_orders['c_bookshelf_norm'].fillna(0)
book_orders['c_visit_norm'] = book_orders['c_visit_norm'].fillna(0)

Умножаю веса

In [25]:
book_orders["c_books_norm"] = 0.1 * book_orders["c_books_norm"]
book_orders["c_ordrs_norm"] = 0.3 * book_orders["c_ordrs_norm"]
book_orders["c_bookmarks_norm"] = 0.25 * book_orders["c_bookmarks_norm"]
book_orders["c_favourites_norm"] = 0.1 * book_orders["c_books_norm"]
book_orders["c_bookshelf_norm"] = 0.15 * book_orders["c_books_norm"]
book_orders["c_books_norm"] = 0.1 * book_orders["c_books_norm"]

In [26]:
book_orders = pd.DataFrame(book_orders.drop(columns = ['c_books', 'c_orders', 'c_bookmarks', 'c_favourites', 'c_bookshelf','c_visit']))

Суммирую все нормированные активности

In [27]:
sum_all = book_orders['c_books_norm'] + book_orders['c_ordrs_norm'] + book_orders['c_bookmarks_norm'] + book_orders['c_ordrs_norm'] + book_orders['c_favourites_norm'] + book_orders['c_bookshelf_norm'] + book_orders['c_visit_norm']
book_orders['sum_all'] = sum_all 

Сортирую по сумме активностей конкретного пользователя

In [71]:

if book_orders.isin(['00003052-D3BC-4965-9239-3BEE91FED0A1']).any().any() :
    book_orders_1 = book_orders.loc[book_orders['ebs_user_id'] == '00003052-D3BC-4965-9239-3BEE91FED0A1']
    book_orders_1 = book_orders_1.sort_values(by='sum_all', ascending=False)
    print(book_orders_1)
else:
    print('Нет в списке')

                             ebs_user_id discipline_id  c_books_norm  \
7   00003052-D3BC-4965-9239-3BEE91FED0A1          4059      0.010000   
12  00003052-D3BC-4965-9239-3BEE91FED0A1          9326      0.001778   
2   00003052-D3BC-4965-9239-3BEE91FED0A1          3148      0.001596   
10  00003052-D3BC-4965-9239-3BEE91FED0A1          8069      0.001596   
5   00003052-D3BC-4965-9239-3BEE91FED0A1          3763      0.001351   
4   00003052-D3BC-4965-9239-3BEE91FED0A1          3761      0.001316   
0   00003052-D3BC-4965-9239-3BEE91FED0A1         12339      0.001277   
3   00003052-D3BC-4965-9239-3BEE91FED0A1          3760      0.001268   
1   00003052-D3BC-4965-9239-3BEE91FED0A1         16478      0.001059   
11  00003052-D3BC-4965-9239-3BEE91FED0A1          8070      0.000968   
9   00003052-D3BC-4965-9239-3BEE91FED0A1          4365      0.000921   
6   00003052-D3BC-4965-9239-3BEE91FED0A1          4045      0.000556   
8   00003052-D3BC-4965-9239-3BEE91FED0A1          4154      0.00